<a href="https://colab.research.google.com/github/ProfSynapse/Toolset-Training/blob/main/kto_colab_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Package Installation and Imports
Install required packages including unsloth and flash-attention, and import necessary libraries for the KTO finetuning process.

In [ ]:
# Install required packages
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# Install Flash Attention 2 for softcapping support
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

# Import necessary libraries
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import os
import re
from typing import List, Literal, Optional
from datasets import load_dataset
from trl import KTOConfig, KTOTrainer
from transformers import TrainingArguments

# Model Loading and Configuration
Load the pre-trained model and tokenizer using FastLanguageModel, and configure basic parameters like sequence length and quantization settings.

In [ ]:
# Model Loading and Configuration

# Set basic parameters
max_seq_length = 4096  # Choose any! We auto support RoPE Scaling internally!
dtype = None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.


# Load the pre-trained model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gpt-oss-20b-unsloth-bnb-4bit",  # Choose ANY! eg mistralai/Mistral-7B-Instruct-v0.2
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    # token="hf_...",  # use one if using gated models like meta-llama/Llama-2-7b-hf
)

# Add proper chat template if missing
if tokenizer.chat_template is None:
    DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"
    tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

==((====))==  Unsloth 2024.12.1: Fast Qwen2 patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


# Dataset Preparation and Processing
Load and prepare the Claudesidian synthetic dataset from Hugging Face Hub. The dataset contains 1,000 ChatML-formatted examples with boolean labels (true=desirable, false=undesirable). These are converted to KTO's chosen/rejected format and paired for contrastive learning.

In [ ]:
# Dataset Preparation and Processing

# Load the Claudesidian synthetic dataset from Hugging Face
raw_datasets = load_dataset(
    "professorsynapse/claudesidian-synthetic-dataset",
    data_files="syngen_toolset_v1.0.0_claude.jsonl"
)
train_dataset = raw_datasets["train"]

print("=== Dataset Info ===")
print(f"Total examples: {len(train_dataset)}")

# KTO trainer expects: prompt, completion, label format
# We need to extract the user prompt and assistant response separately

def prepare_kto_format(example):
    """
    Convert ChatML format to KTO format.
    
    KTO requires:
    - prompt: the user input
    - completion: the assistant response
    - label: True for desirable, False for undesirable
    """
    conversations = example["conversations"]
    label = example["label"]
    
    # Extract user message (prompt) and assistant message (completion)
    user_msgs = [msg for msg in conversations if msg["role"] == "user"]
    assistant_msgs = [msg for msg in conversations if msg["role"] == "assistant"]
    
    if not user_msgs or not assistant_msgs:
        return None
    
    # Get the first user message as prompt
    prompt = user_msgs[0]["content"]
    
    # Get the first assistant message as completion
    completion = assistant_msgs[0]["content"]
    
    return {
        "prompt": prompt,
        "completion": completion,
        "label": label  # True = desirable, False = undesirable
    }

# Process all examples
processed_dataset = []
for example in train_dataset:
    result = prepare_kto_format(example)
    if result is not None:
        processed_dataset.append(result)

print(f"Processed examples: {len(processed_dataset)} (filtered {len(train_dataset) - len(processed_dataset)} invalid)")

# Count distribution
chosen_count = sum(1 for ex in processed_dataset if ex["label"] == True)
rejected_count = sum(1 for ex in processed_dataset if ex["label"] == False)

print(f"\nDataset distribution:")
print(f"  Desirable (True): {chosen_count}")
print(f"  Undesirable (False): {rejected_count}")
print(f"  Ratio: {chosen_count / rejected_count:.2f}:1")

# Create HuggingFace dataset in correct KTO format
from datasets import Dataset as HFDataset
train_subset = HFDataset.from_dict({
    "prompt": [ex["prompt"] for ex in processed_dataset],
    "completion": [ex["completion"] for ex in processed_dataset],
    "label": [ex["label"] for ex in processed_dataset],
})

print(f"\nFinal KTO dataset: {len(train_subset)} examples")
print(f"Columns: {train_subset.column_names}")
print(f"\nSample example:")
print(f"  Prompt: {train_subset['prompt'][0][:100]}...")
print(f"  Completion: {train_subset['completion'][0][:100]}...")
print(f"  Label: {train_subset['label'][0]}")

# Model Training Setup
Configure the LoRA adapters and set up the KTO trainer with appropriate training arguments. The trainer uses the paired Claudesidian dataset (742 desirable/254 undesirable examples) for contrastive learning.

In [ ]:
# Model Training Setup

# Configure the LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)

# Prepare model for training
model.config.use_cache = False

# Initialize KTO trainer - simple configuration for debugging
print("Initializing KTO trainer...")
kto_trainer = KTOTrainer(
    model=model,
    args=KTOConfig(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        num_train_epochs=1,
        learning_rate=5e-7,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        output_dir="outputs",
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        warmup_ratio=0.1,
        seed=42,
        report_to="none",
        remove_unused_columns=False,
        max_grad_norm=1.0,
        dataloader_num_workers=0,
        beta=0.1,
        desirable_weight=1.0,
        undesirable_weight=1.0,
    ),
    train_dataset=train_subset,
    processing_class=tokenizer,
)

print("✓ KTO trainer initialized successfully")
print(f"Dataset size: {len(train_subset)} examples")
print(f"Per-device batch size: 2")
print(f"Gradient accumulation steps: 2")
print(f"Effective batch size: 4")

# Training Execution
Execute the training process with the configured trainer and monitor the training progress.

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
1.709 GB of memory reserved.


In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

print("=== KTO Training Debug ===\n")

# First, let's understand what the trainer is doing
print("1. Dataset Info:")
print(f"   Total examples: {len(train_subset)}")
print(f"   Columns: {train_subset.column_names}")
print(f"   Prompt column type: {type(train_subset['prompt'][0])}")
print(f"   Completion column type: {type(train_subset['completion'][0])}")
print(f"   Label column type: {type(train_subset['label'][0])}")

# Check label distribution
label_true = sum(1 for x in train_subset['label'] if x == True)
label_false = sum(1 for x in train_subset['label'] if x == False)
print(f"\n   Label distribution:")
print(f"     True (desirable): {label_true}")
print(f"     False (undesirable): {label_false}")

# Check text lengths
print(f"\n2. Text Length Analysis:")
prompt_lens = [len(p) for p in train_subset['prompt']]
completion_lens = [len(c) for c in train_subset['completion']]
print(f"   Prompt lengths - Min: {min(prompt_lens)}, Max: {max(prompt_lens)}, Avg: {sum(prompt_lens)/len(prompt_lens):.0f}")
print(f"   Completion lengths - Min: {min(completion_lens)}, Max: {max(completion_lens)}, Avg: {sum(completion_lens)/len(completion_lens):.0f}")

# Test tokenization
print(f"\n3. Manual Tokenization Test:")
sample_prompts = train_subset['prompt'][:2]
sample_completions = train_subset['completion'][:2]

prompt_tokens = tokenizer(
    sample_prompts,
    padding=True,
    truncation=True,
    max_length=4096,
    return_tensors="pt"
)
print(f"   Sample prompts tokenized shape: {prompt_tokens['input_ids'].shape}")

completion_tokens = tokenizer(
    sample_completions,
    padding=True,
    truncation=True,
    max_length=4096,
    return_tensors="pt"
)
print(f"   Sample completions tokenized shape: {completion_tokens['input_ids'].shape}")

# Check trainer config
print(f"\n4. Trainer Configuration:")
print(f"   Batch size: {kto_trainer.args.per_device_train_batch_size}")
print(f"   Gradient accumulation: {kto_trainer.args.gradient_accumulation_steps}")
print(f"   Max sequence length: {kto_trainer.args.max_length if hasattr(kto_trainer.args, 'max_length') else 'default'}")
print(f"   Beta: {kto_trainer.args.beta if hasattr(kto_trainer.args, 'beta') else 'N/A'}")
print(f"   Remove unused columns: {kto_trainer.args.remove_unused_columns}")

# Now attempt training
print(f"\n5. Attempting Training:")
print(f"   Starting train()...\n")

try:
    kto_trainer.train()
    print("\n✓ Training completed successfully!")
except Exception as e:
    print(f"\n✗ ERROR: {type(e).__name__}")
    print(f"   Message: {e}")
    import traceback
    print("\n   Full traceback:")
    traceback.print_exc()
    
    # Additional debugging
    print("\n6. Additional Debug Info:")
    print(f"   Dataset has {len(train_subset)} examples")
    print(f"   First example keys: {list(train_subset[0].keys())}")
    print(f"   First example values: prompt={train_subset[0]['prompt'][:50]}..., completion={train_subset[0]['completion'][:50]}..., label={train_subset[0]['label']}")

# Model Saving and Export
Save the trained model in different formats including LoRA adapters and merged model.

In [ ]:
# Model Saving and Export

# Local saving
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

# Save merged model as float16 or int4
if False: # Set to True to save
    model.save_pretrained_merged("merged_model", tokenizer, save_method = "merged_16bit")
    # model.save_pretrained_merged("merged_model", tokenizer, save_method = "merged_4bit")
    # model.save_pretrained_merged("merged_model", tokenizer, save_method = "lora")

# Save to HuggingFace Hub
if False: # Set to True to save
    model.push_to_hub_merged("your_name/model", tokenizer, save_method = "merged_16bit", token = "...")
    # save_method can be "merged_16bit", "merged_4bit", or "lora"

# Save to GGUF format (for llama.cpp)
if False: # Set to True to save
    from transformers import AutoTokenizer
    model.save_pretrained_merged("merged_model", tokenizer, save_method = "merged_16bit")
    !git clone https://github.com/ggerganov/llama.cpp
    !cd llama.cpp && make
    !python3 llama.cpp/convert.py merged_model/ --outfile model-unsloth.gguf
    # Also supports quantization
    !./llama.cpp/quantize model-unsloth.gguf model-unsloth-Q4_K_M.gguf Q4_K_M

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml",
    mapping = {"role": "role", "content": "content", "user": "user", "assistant": "assistant"},
)

FastLanguageModel.for_inference(model)

def generate_response(message):
    print("\n" + "="*60 + "\nQUESTION:\n" + "="*60)
    print(message + "\n")
    print("-"*60 + "\nRESPONSE:\n" + "-"*60)

    messages = [{"content": message, "role": "user"}]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True,
        return_tensors = "pt"
    ).to("cuda")

    from transformers import TextStreamer
    text_streamer = TextStreamer(tokenizer, skip_special_tokens=True, skip_prompt=True)
    outputs = model.generate(
        input_ids = inputs,
        streamer = text_streamer,
        temperature = 0.1,
        max_new_tokens = 1024,
        use_cache = True
    )
    return outputs

# Test questions - Claudesidian vault operations
questions = [
    # Test 1: Basic content reading scenario
    "I need to review my meeting notes from yesterday. Can you help me find and read the notes?",
    
    # Test 2: Multi-step workflow with workspace context
    "I'm switching to my 'Q4-Planning' workspace. Once switched, create a summary document that lists all my project notes and their status.",
    
    # Test 3: Folder operations and organization
    "My notes are getting disorganized. Rename the 'old-drafts' folder to 'archive-2024' and then create a README.md file inside it explaining its purpose.",
    
    # Test 4: Search and cross-workspace coordination
    "Search across all my workspaces for notes containing 'roadmap' or 'strategy'. After finding them, create a unified index file that links to all results.",
    
    # Test 5: Error handling and recovery
    "I want to create a backup of an important note, but I'm not sure what the exact file path is. Help me find it and then create a backup copy.",
]

# Generate responses
for i, question in enumerate(questions, 1):
    print(f"\n\n{'='*60}\nTEST CASE {i}: Claudesidian Tool Use\n{'='*60}")
    generate_response(question)